1. SSD 네트워크 모델 구축

2. DBox 구현

In [1]:
from math import sqrt
from itertools import product

import pandas as pd
import torch
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

## 네트워크 개요

1. 입력 이미지는 첫번 째로 VGG 모듈을 통과.
  
  - 총 10회의 convolution 연산을 수행한 feature map (conv4_3)은 별도로 추출하여 L2 정규화 적용. (source 1, 38x38x512)

2. vgg 모듈의 최종 output feature map은 19x19x1024 (source 2)

3. source2를 extra 모듈의 입력으로 제공. 총 8회의 convolution 연산 중 2회 마다 중간 결과 feature map을 source 3~6으로 지칭 (10x10x512, 5x5x256, 3x3x256, 1x1x256|)

4. loc 모듈은 source1~6을 각각 입력으로 받아 개별적으로 1회씩 convolution 연산 수행 -> 8732개의 Dbox offset 정보 반환

5. conf 모듈은 source1~6을 각각 입력으로 받아 개별적으로 1회씩 convolution 연산을 수행 -> 8732개 Dbox에 대한 클래스 신뢰도 반환

## 1. VGG 모듈



In [2]:
def make_vgg():
    layers = []
    in_channels = 3
    
    # vgg 모듈이 사용하는 합성곱 층, 맥스 풀링 채널 수 지정
    cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'MC',
           512, 512, 512, 'M', 512, 512, 512]

    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'MC':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6,
               nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    
    return nn.ModuleList(layers)

In [3]:
vgg_test = make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

## 2. extra

In [4]:
def make_extras():
    layers = []
    in_channels = 1024  # vgg 모듈 최종 feature map은 1024 채널 (source2의 채널)

    # 합성곱 layer 채널수 설정
    # 2개의 convolution block마다 source 출력 (1x1 conv, 3x3 conv로 구성된 bottlencek과 유사한 구조)
    cfg = [256, 512, 128, 256, 128, 256, 128, 256]
    
    # source3
    layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]  # 1024 -> 256
    layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)]  # 256 -> 512

    # source4
    layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]  # 512 -> 128
    layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)]  # 128 -> 256

    # source5
    layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]  # 256 -> 128
    layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))]  # 128 -> 256

    # source6
    layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]  # 256 -> 128
    layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))]  # 128 -> 256

    return nn.ModuleList(layers)

In [5]:
extras_test = make_extras()
print(extras_test)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


`source1` : 38 x 38 x 512

`source2` : 19 x 19 x 1024

`source3` : 10 x 10 x 512

`source4` : 5 x 5 x 256

`source5` : 3 x 3 x 256

`source6` : 1 x 1 x 256

## 3. loc & conf

loc는 DBox의 offset 정보 출력

conf는 각 클래스에 대한 신뢰도 출력

In [6]:
# bbox_aspect_num : source별로 개별 feature map에 존재하는 DBox의 종류
# ex) source1 : 38x38x4 (각 픽셀 당 4개의 DBox)  /  source2 :19x19x6 (각 픽셀 당 6개 DBox)
def make_loc_conf(num_classes=21, bbox_aspect_num=[4, 6, 6, 6, 4, 4]):

    loc_layers = []
    conf_layers = []
    
    # source1 처리
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[0] * 4, kernel_size=3, padding=1)]  # DBox의 좌표 정보
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[0] * num_classes, kernel_size=3, padding=1)]  # 21개 클래스에 대한 DBox 신뢰도

    # VGG 모듈 최종 output(source2) 처리
    loc_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * num_classes, kernel_size=3, padding=1)]
    
    ## extra outputs
    # source3 처리
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[2] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[2] * num_classes, kernel_size=3, padding=1)]

    # source4 처리
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[3] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[3] * num_classes, kernel_size=3, padding=1)]

    # source5 처리
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[4] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[4] * num_classes, kernel_size=3, padding=1)]

    # source6 처리
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[5] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[5] * num_classes, kernel_size=3, padding=1)]

    return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)


In [7]:
# 동작 확인
loc_test, conf_test = make_loc_conf()
print(loc_test)
print(conf_test)

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


## 4. L2 Norm

source1 feature map의 통계적 특성을 채널을 기준으로 정규화

 - 각 채널 마다 38x38=1444개의 픽셀이 존재
 - 동일한 위치의 픽셀들에 대해 채널 방향으로 정규화

1444개 픽셀 마다 채널 방향으로(512 dim) 제곱합을 구하고 루트

In [8]:
class L2Norm(nn.Module):
    def __init__(self, input_channels=512, scale=20):
        super(L2Norm, self).__init__()  
        self.weight = nn.Parameter(torch.Tensor(input_channels))
        self.scale = scale 
        self.reset_parameters()  # 파라미터의 초기화
        self.eps = 1e-10

    def reset_parameters(self):
        init.constant_(self.weight, self.scale)  # weight값이 모두 20으로 초기화됨

    def forward(self, x):
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt() + self.eps  # 채널 방향으로 제곱합, 루트 계산
        x = torch.div(x, norm)

        weights = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
        out = weights * x

        return out

In [9]:
# 참고
x = torch.randn(1, 512, 38, 38)
print(x.pow(2).sum(dim=1, keepdim=True).shape)
weight = nn.Parameter(torch.Tensor(512))
print(weight.shape)
print(weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).shape)
print(weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x).shape)

torch.Size([1, 1, 38, 38])
torch.Size([512])
torch.Size([1, 512, 1, 1])
torch.Size([1, 512, 38, 38])


## 5. DBox 구현

4개인 경우 : 작은 정사각형, 큰 정사각형, 1:2 직사각형, 2:1 직사각형

6개인 경우 : 3:1 직사각형, 1:3 직사각형 추가

In [10]:
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()

        # Config
        self.image_size = cfg['input_size']  # 이미지 크기 : 300
        self.feature_maps = cfg['feature_maps']  # [38, 19, 10, 5, 3, 1] 각 source별 feature map 크기
        self.num_priors = len(cfg["feature_maps"])  # source 개수 6

        self.steps = cfg['steps']  # [8, 16, 32, 64, 100, 300] DBox 픽셀 크기 (source1 : 작은 객체 인식 용이 / source6 : 큰 객체 인식 용이)    
        self.min_sizes = cfg['min_sizes']  # 작은 정사각형의 DBox 크기
        self.max_sizes = cfg['max_sizes']  # 큰 정사각형의 DBox 크기
        self.aspect_ratios = cfg['aspect_ratios']  # source별 정,직사각형 DBox 종횡비

    def make_dbox_list(self):
        mean = []
        for k, f in enumerate(self.feature_maps):
            # fxf개의 좌상단 좌표 정보
            for i, j in product(range(f), repeat=2):
                # source 별 feature map의 크기
                f_k = self.image_size / self.steps[k]  # 300 / 'steps': [8, 16, 32, 64, 100, 300]
                
                # DBox 중심 좌표. 0~1로 정규화함
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # 작은 정사각형 DBox [cx, cy, w, h] - 중심점만 달라지고 w, h는 동일
                s_k = self.min_sizes[k] / self.image_size
                mean += [cx, cy, s_k, s_k]

                # 큰 정사각형 DBox [cx, cy, w, h]
                s_k_prime = sqrt(s_k * (self.max_sizes[k] / self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                # 종횡비가 다른 직사각형 DBox
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k * sqrt(ar), s_k / sqrt(ar)]
                    mean += [cx, cy, s_k / sqrt(ar), s_k * sqrt(ar)]

        output = torch.Tensor(mean).view(-1, 4)  # (8732, 4)
        output.clamp_(max=1, min=0)  # DBox가 이미지 밖에 위치하지 않도록

        return output

In [11]:
ssd_cfg = {
    'num_classes': 21,  # 총 클래스 수 (배경 포함)
    'input_size': 300,  # 이미지 크기
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # source 별 DBox 종류
    'feature_maps': [38, 19, 10, 5, 3, 1],  # source 별 feature map 크기
    'steps': [8, 16, 32, 64, 100, 300],  # source 별 DBox 크기
    'min_sizes': [30, 60, 111, 162, 213, 264],  # 작은 정사각형 DBox 크기
    'max_sizes': [60, 111, 162, 213, 264, 315],  # 큰 정사각형 DBox 크기
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],  # 직사각형 DBox용 (4개인 경우 2만 이용, 6개인 경우 1:3, 3:1 비율 직사각형이 추가되기 때문에 [2, 3])
}

In [12]:
# DBox 
dbox = DBox(ssd_cfg)
dbox_list = dbox.make_dbox_list()

# DBox 출력 확인
pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
...,...,...,...,...
8727,0.833333,0.833333,0.502046,1.000000
8728,0.500000,0.500000,0.880000,0.880000
8729,0.500000,0.500000,0.961249,0.961249
8730,0.500000,0.500000,1.000000,0.622254


In [13]:
class SSD(nn.Module):
    
    def __init__(self, phase, cfg):
        super(SSD, self).__init__()

        self.phase = phase  
        self.num_classes = cfg["num_classes"]  

        # SSD 네트워크 구성
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg["num_classes"], cfg["bbox_aspect_num"])

        # 8792개 DBox 생성
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()

        if phase == 'inference':
            self.detect = Detect()

In [14]:
# 동작 확인
ssd_test = SSD(phase="train", cfg=ssd_cfg)
print(ssd_test)

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

## 6. Forward 

### 6.1 decode

Inference 용

DBox, offset 정보를 이용하여 BBox 좌표 정보를 생성

In [15]:
def decode(loc, dbox_list):
    """
     loc:  [8732, 4]  [Δcx, Δcy, Δw, Δh]
        SSD 모델을 통해 얻은 오프셋 정보.
    dbox_list: [8732, 4]  [cx, cy, w, h]
        DBox 정보
    """
    boxes = torch.cat((
        dbox_list[:, :2] + loc[:, :2] * 0.1 * dbox_list[:, 2:],  # BBox cx, cy 계산
        dbox_list[:, 2:] * torch.exp(loc[:, 2:] * 0.2)), dim=1)  # w, h 계산
    
    # [cx, cy, w, h] -> [xmin, ymin, xmax, ymax]
    boxes[:, :2] -= boxes[:, 2:] / 2 
    boxes[:, 2:] += boxes[:, :2]

    return boxes
    

### 6.2 NMS

8732개의 DBox를 사용하여 객체를 탐지하기 때문에 이미지 내 동일한 객체에 조금씩 다른 예측 BBox가 다수 피팅되는 경우 존재.

BBox끼리 공통되는 면적이 ths 이상일 때 중복 BBox로 판정하고, 그 중 신뢰도가 가장 높은 BBox만 남기고 나머지는 삭제 (하나의 객체, 하나의 BBox)

In [35]:
def nm_suppression(boxes, scores, overlap=0.45, top_k=200):
    """
    boxes 중에서 겹치는(overlap 이상)의 BBox를 삭제

    Parameters
    ----------
    boxes : [신뢰도 임계값(0.01)을 넘은 BBox 수, 4]
        BBox 좌표 정보
    scores :[신뢰도 임계값(0.01)을 넘은 DBox 수]
        conf 정보

    Returns
    -------
    keep : 리스트
        신뢰도의 내림차순으로 nms를 통과한 index 저장
    count: int
        nms를 통과한 BBox 수
    """
    # 초기화
    keep = scores.new(scores.size(0)).zero_().long()  # [신뢰도 ths를 넘은 BBox 수]
    count = 0

    # 개별 BBox의 면적 계산
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = torch.mul(x2 - x1, y2 - y1)
    
    # boxes 복사 (iou 계산시 활용)
    tmp_x1 = boxes.new()  # tensor([])
    tmp_y1 = boxes.new()
    tmp_x2 = boxes.new()
    tmp_y2 = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()

    # 신뢰도 기준으로 정렬 (오름차순)
    # v : 오름차순으로 정렬된 score 리스트 
    v, idx = scores.sort(0)
    
    # 상위 200개의 BBox index 추출
    idx = idx[-top_k:]

    while idx.numel() > 0:
        i = idx[-1]  # 신뢰도가 최대인 BBox index부터 추출

        keep[count] = i  # keep 리스트 앞 부분 부터 신뢰도가 최대인 index 저장
        count += 1

        if idx.size(0) == 1:
           break
        
        # keep에 저장된 index는 idx 리스트에서 삭제
        # keep에 저장된 index의 BBox를 기준으로 idx의 나머지 BBox들과 겹치는 면적 계산, 삭제
        idx = idx[:-1]  

        ## keep에 저장된 BBox와 겹치는 정도가 큰 BBox를 추출하여 삭제
        # idx의 BBox 정보를, out으로 지정한 변수로 저장
        # index_select 참고 : https://runebook.dev/ko/docs/pytorch/generated/torch.index_select
        torch.index_select(x1, 0, idx, out=tmp_x1)
        torch.index_select(y1, 0, idx, out=tmp_y1)
        torch.index_select(x2, 0, idx, out=tmp_x2)
        torch.index_select(y2, 0, idx, out=tmp_y2)
        
        # 폭, 높이 계산
        tmp_w.resize_as_(tmp_x2)
        tmp_h.resize_as_(tmp_y2)
        tmp_w = tmp_x2 - tmp_x1
        tmp_h = tmp_y2 - tmp_y1

        # w, h가 음수인 경우 0으로
        tmp_w = torch.clamp(tmp_w, min=0.0)
        tmp_h = torch.clamp(tmp_h, min=0.0)

        # 면적 계산
        inter = tmp_w * tmp_h
        
        # iou 계산 (broadcasting됨)
        rem_areas = torch.index_select(area, 0, idx)  # 각 BBox의 원래 면적
        union = (rem_areas - inter) + area[i]  # 두 구역의 합(OR)의 면적
        IoU = inter / union
        
        # iou가 ths(overlap)보다 작은 idx만 남김
        idx = idx[IoU.le(overlap)]  # le : less than or equal to 연산 수행

    return keep, count

### 6.3 Detect

입력 :                  
1. loc 모듈 출력 : offset 정보 (B, 8732, 4)
2. conf 모듈 출력 : (B, 8732, 21) - softmax 적용
3. DBox 정보 : (8732, 4)


출력 : (B, 21, 200, 5) - 객체 탐지 결과인 BBox

 - 배치, 클래스, 신뢰도 상위 200개 BBox, (신뢰도, 좌표 정보)


forward

1. 6.1 decode를 이용하여 DBox 정보, offset 정보를 BBox로 변환
2. 신뢰도가 ths 이상인 BBox만 추출
3. NMS로 중복 BBox 제거

In [ ]:
class Detect(Function):

    def __init__(self, conf_thresh=0.01, top_k=200, nms_thresh=0.45):
        self.softmax = nn.Softmax(dim=-1)  # 신뢰도를 정규화
        self.conf_thresh = conf_thresh  # conf가 conf_thresh=0.01보다 높은 DBox만 추출
        self.top_k = top_k  # nms로 신뢰도가 높은 top_k개의 계산에 사용
        self.nms_thresh = nms_thresh  # nms에서 iou가 nms_thresh=0.45보다 크면 동일한 물체의 BBox로 간주하고 삭제

    def forward(self, loc_data, conf_data, dbox_list):
        """
        Parameters
        ----------
        loc_data:  [batch_num,8732,4]
            오프셋 정보
        conf_data: [batch_num, 8732,num_classes]
            감지 신뢰도
        dbox_list: [8732,4]
            DBox의 정보

        Returns
        -------
        output : torch.Size([batch_num, 21, 200, 5])
            (batch_num, 클래스, conf의 top200, BBox 정보)
        """
        num_batch = loc_data.size(0)  
        num_dbox = loc_data.size(1) 
        num_classes = conf_data.size(2)  

        conf_data = self.softmax(conf_data)
        conf_preds = conf_data.transpose(2, 1)

        output = torch.zeros(num_batch, num_classes, self.top_k, 5)

        for i in range(num_batch):
            # 1. # offset정보로 DBox를 변경하여 예측 BBox 좌표 계산
            decoded_boxes = decode(loc_data[i], dbox_list)        
            conf_scores = conf_preds[i].clone()
            
            # 클래스 별로 loop
            for cl in range(1, num_classes):
                # 2. ths 이상의 신뢰도를 갖는 BBox index추출
                c_mask = conf_scores[cl].gt(self.conf_thresh)  # greater than (넘으면 1, 아니면 0), (8732,)
                scores = conf_scores[cl][c_mask]  # [ths를 넘은 BBox 수]

                if scores.nelement() == 0:  # ths를 넘는 경우가 없으면
                    continue
                
                # 임계값 이상의 신뢰도를 갖는 BBox만
                l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)  # (8732, 4)
                boxes = decoded_boxes[l_mask].view(-1, 4)  # (ths를 넘은 BBox 수, 4)

                # 3. NMS -> 중복 제거
                # ids : 신뢰도 내림차순으로 nms를 통과한 index / count : nms를 통과한 BBox 수
                ids, count = nm_suppression(boxes, scores, self.nms_thresh, self.top_k)
                output[i, cl, :count] = torch.cat((scores[ids[:count]].unsqueeze(1),
                                                   boxes[ids[:count]]), 1)
                
        return output  # torch.Size([1, 21, 200, 5])

## 7. 최종 SSD 클래스 구현


1. vgg, extra 모듈을 통해 source1~6 출력

2. source1~6에 loc, conf 모듈을 적용시켜 offset, 신뢰도 정보 추출
  - offset loc : (B, 8732, 4)
  - conf : (B, 8732, 21)

3. 학습 시에는 output : (loc, conf, 8732, 4), inference 시에는 (B, 21, 200, 5)

In [ ]:
class SSD(nn.Module):
    def __init__(self, phase, cfg):
        super(SSD, self).__init__()

        self.phase = phase
        self.num_classes = cfg['num_classes']

        # SSD 네트워크
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg['num_classes'], cfg['bbox_aspect_num'])
        
        # DBox 생성 (8732, 4)
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        # inference시에는 detect 호출
        if phase == 'inference':
            self.detect = Detect()

    def forward(self, x):
        sources = []
        loc = []
        conf = []
        
        # source1 (vgg conv4_3까지)
        for k in range(23):
            x = self.vgg[k](x)

        source1 = self.L2Norm(x)
        sources.append(source1)

        # source2 (vgg를 끝까지 계산)
        for k in range(23, len(self.vgg)):
            x = self.vgg[k](x)

        sources.append(x)
        
        # source3~6 (extras 모듈 - 총 8회의 합성곱)
        for k, v in enumerate(self.extras):
            x = F.relu(v(x), inplace=True)
            if k % 2 == 1:  # 2회 합성곱 마다 중간 출력 결과를 source로 저장
                sources.append(x)

        for (x, l, c) in zip(sources, self.loc, self.conf):
            loc.append(l(x).permute(0, 2, 3, 1).contiguous())  # l(x) : (B, 4x화면 비 종류, h, w), source마다 화면 비 다름
            conf.append(c(x).permute(0, 2, 3, 1).contiguous())  # c(x) : (B, 21x화면 비 종류, h, w)

        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)  # (B, 8732x4)
        conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)  # (B, 8732x21)

        loc = loc.view(loc.size(0), -1, 4)  # (B, 8732x4) -> (B, 8732, 4)
        conf = conf.view(conf.size(0), -1, self.num_classes)  # (B, 8732x21) -> (B, 8732, 21)

        output = (loc, conf, self.dbox_list)    

        if self.phase == 'inference':
            return self.detect(output[0], output[1], output[2])

        else:
          return output